In [1]:
# import osmnx as ox
import pandas as pd
import geopandas as gpd
import networkx as nx

import warnings
import sys
import os
warnings.filterwarnings("ignore")
sys.stderr = open(os.devnull, 'w')

# local crs
local_crs = 32636

In [7]:
adj_mx = pd.read_pickle('/Users/mvin/Code/PopFrame/examples/data/model_data/adj_mx.pickle')
towns = gpd.read_parquet('/Users/mvin/Code/PopFrame/examples/data/model_data/towns.parquet')
okrugs = gpd.read_parquet('/Users/mvin/Code/PopFrame/examples/data/model_data/okrugs.parquet')
rayons = gpd.read_parquet('/Users/mvin/Code/PopFrame/examples/data/model_data/rayons.parquet')
territory = gpd.read_parquet('/Users/mvin/Code/PopFrame/examples/data/model_data/territory.parquet')


In [8]:
from popframe.models.region import Region

region = Region(
  towns=towns,
  okrugs=okrugs, 
  rayons=rayons, 
  adjacency_matrix=adj_mx,
  territory=territory
)

In [9]:
region.territory

,geometry,name
0,"MULTIPOLYGON (((329788.882 6583309.434, 330250...",Сиверский


In [41]:
def calculate_potential(criteria_values):
    profiles = {
        "Жилая застройка - ИЖС": {
            "criteria": {"Население": 1, "Транспорт": 2, "Экология": 4, "Соц-об": 4, "Инж инф": 3, "Кул-дос-сп": 0},
            "weights": {"Население": 2, "Транспорт": 1, "Экология": 1, "Соц-об": 1, "Инж инф": 1, "Кул-дос-сп": 1}
        },
        "Жилая застройка - Малоэтажная": {
            "criteria": {"Население": 3, "Транспорт": 3, "Экология": 4, "Соц-об": 5, "Инж инф": 4, "Кул-дос-сп": 1},
            "weights": {"Население": 1, "Транспорт": 1, "Экология": 1, "Соц-об": 1, "Инж инф": 1, "Кул-дос-сп": 1}
        },
        "Жилая застройка - Среднеэтажная": {
            "criteria": {"Население": 4, "Транспорт": 4, "Экология": 4, "Соц-об": 4, "Инж инф": 5, "Кул-дос-сп": 2},
            "weights": {"Население": 1, "Транспорт": 1, "Экология": 1, "Соц-об": 1, "Инж инф": 1, "Кул-дос-сп": 1}
        },
        "Жилая застройка - Многоэтажная": {
            "criteria": {"Население": (4, 5), "Транспорт": 5, "Экология": 4, "Соц-об": 5, "Инж инф": 5, "Кул-дос-сп": 3},
            "weights": {"Население": 1, "Транспорт": 1, "Экология": 1, "Соц-об": 1, "Инж инф": 1, "Кул-дос-сп": 1}
        },
        "Общественно-деловая": {
            "criteria": {"Население": 4, "Транспорт": 5, "Экология": 4, "Соц-об": 2, "Инж инф": 5, "Кул-дос-сп": 3},
            "weights": {"Население": 1, "Транспорт": 1, "Экология": 1, "Соц-об": 1, "Инж инф": 1, "Кул-дос-сп": 1}
        },
        "Рекреационная": {
            "criteria": {"Население": 0, "Транспорт": 0, "Экология": 0, "Соц-об": 0, "Инж инф": 0, "Кул-дос-сп": 0},
            "weights": {"Население": 1, "Транспорт": 1, "Экология": 1, "Соц-об": 1, "Инж инф": 1, "Кул-дос-сп": 1}
        },
        "Специального назначения - медицинский центр": {
            "criteria": {"Население": 4, "Транспорт": 4, "Экология": 5, "Соц-об": 0, "Инж инф": 5, "Кул-дос-сп": 0},
            "weights": {"Население": 1, "Транспорт": 1, "Экология": 1, "Соц-об": 1, "Инж инф": 1, "Кул-дос-сп": 1}
        },
        "Специального назначения - туристический кластер": {
            "criteria": {"Население": 3, "Транспорт": 3, "Экология": 5, "Соц-об": 1, "Инж инф": 3, "Кул-дос-сп": 0},
            "weights": {"Население": 1, "Транспорт": 1, "Экология": 1, "Соц-об": 1, "Инж инф": 1, "Кул-дос-сп": 1}
        },
        "Промышленная": {
            "criteria": {"Население": (3, 4), "Транспорт": 4, "Экология": 0, "Соц-об": 1, "Инж инф": 4, "Кул-дос-сп": 2},
            "weights": {"Население": 1, "Транспорт": 1, "Экология": 1, "Соц-об": 1, "Инж инф": 1, "Кул-дос-сп": 1}
        },
        "Сельско-хозяйственная": {
            "criteria": {"Население": (2, 3), "Транспорт": 3, "Экология": (4, 5), "Соц-об": 3, "Инж инф": (2, 3), "Кул-дос-сп": 2},
            "weights": {"Население": 1, "Транспорт": 1, "Экология": 1, "Соц-об": 1, "Инж инф": 1, "Кул-дос-сп": 1}
        },
        "Транспортная инженерная": {
            "criteria": {"Население": (1, 2), "Транспорт": 0, "Экология": 0, "Соц-об": 0, "Инж инф": 2, "Кул-дос-сп": 0},
            "weights": {"Население": 1, "Транспорт": 1, "Экология": 1, "Соц-об": 1, "Инж инф": 1, "Кул-дос-сп": 1}
        }
    }

    def is_criterion_satisfied(profile_value, criterion_value):
        if isinstance(profile_value, tuple):
            return profile_value[0] <= criterion_value <= profile_value[1]
        return criterion_value >= profile_value

    def calculate_exceedance(profile_value, criterion_value):
        if isinstance(profile_value, tuple):
            if profile_value[0] <= criterion_value <= profile_value[1]:
                return criterion_value - profile_value[0]
            return 0
        return max(0, criterion_value - profile_value)

    potential_scores = {}
    for profile, data in profiles.items():
        criteria = data["criteria"]
        weights = data["weights"]
        potential = sum(
            is_criterion_satisfied(criteria.get(criterion, -1), value) * weights.get(criterion, 1)
            for criterion, value in criteria_values.items()
        )
        exceedance = sum(
            calculate_exceedance(criteria.get(criterion, -1), value) * weights.get(criterion, 1)
            for criterion, value in criteria_values.items()
        )
        potential_scores[profile] = (potential, exceedance)

    # Сортировка: сначала по потенциалу и превышению, затем "Рекреационная" в конец, но выше нулевого потенциала
    ranked_profiles = sorted(potential_scores.items(), key=lambda x: (x[0] != "Рекреационная", x[1][0], x[1][1]), reverse=True)
    ranked_profiles = [item for item in ranked_profiles if item[1][0] > 0] + [item for item in ranked_profiles if item[1][0] == 0]
    
    return ranked_profiles

# Пример использования:
criteria_values = {
    "Население": 4,
    "Транспорт": 3,
    "Экология": 5,
    "Соц-об": 4,
    "Инж инф": 4,
    "Кул-дос-сп": 3
}

# criteria_values = {
#     "Население": 0,
#     "Транспорт": 1,
#     "Экология": 0,
#     "Соц-об": 1,
#     "Инж инф": 0,
#     "Кул-дос-сп": 1
# }

ranked_profiles = calculate_potential(criteria_values)
for profile, (potential, exceedance) in ranked_profiles:
    print(f"Профиль: {profile}, Потенциал: {potential}, Превышение: {exceedance}")


Профиль: Жилая застройка - ИЖС, Потенциал: 7, Превышение: 12
Профиль: Специального назначения - туристический кластер, Потенциал: 6, Превышение: 8
Профиль: Транспортная инженерная, Потенциал: 5, Превышение: 17
Профиль: Промышленная, Потенциал: 5, Превышение: 10
Профиль: Жилая застройка - Малоэтажная, Потенциал: 5, Превышение: 4
Профиль: Специального назначения - медицинский центр, Потенциал: 4, Превышение: 7
Профиль: Общественно-деловая, Потенциал: 4, Превышение: 3
Профиль: Сельско-хозяйственная, Потенциал: 4, Превышение: 3
Профиль: Жилая застройка - Среднеэтажная, Потенциал: 4, Превышение: 2
Профиль: Жилая застройка - Многоэтажная, Потенциал: 3, Превышение: 1
Профиль: Рекреационная, Потенциал: 6, Превышение: 23


In [44]:
def calculate_potential(criteria_values):
    profiles = {
        "Жилая застройка - ИЖС": {
            "criteria": {"Население": 1, "Транспорт": 2, "Экология": 4, "Соц-об": 4, "Инж инф": 3, "Кул-дос-сп": 0},
            "weights": {"Население": 1, "Транспорт": 1, "Экология": 3, "Соц-об": 3, "Инж инф": 2, "Кул-дос-сп": 1}
        },
        "Жилая застройка - Малоэтажная": {
            "criteria": {"Население": 3, "Транспорт": 3, "Экология": 4, "Соц-об": 5, "Инж инф": 4, "Кул-дос-сп": 1},
            "weights": {"Население": 2, "Транспорт": 2, "Экология": 3, "Соц-об": 4, "Инж инф": 3, "Кул-дос-сп": 1}
        },
        "Жилая застройка - Среднеэтажная": {
            "criteria": {"Население": 4, "Транспорт": 4, "Экология": 4, "Соц-об": 4, "Инж инф": 5, "Кул-дос-сп": 2},
            "weights": {"Население": 3, "Транспорт": 3, "Экология": 3, "Соц-об": 3, "Инж инф": 4, "Кул-дос-сп": 1}
        },
        "Жилая застройка - Многоэтажная": {
            "criteria": {"Население": (4, 5), "Транспорт": 5, "Экология": 4, "Соц-об": 5, "Инж инф": 5, "Кул-дос-сп": 3},
            "weights": {"Население": 4, "Транспорт": 4, "Экология": 3, "Соц-об": 4, "Инж инф": 4, "Кул-дос-сп": 2}
        },
        "Общественно-деловая": {
            "criteria": {"Население": 4, "Транспорт": 5, "Экология": 4, "Соц-об": 2, "Инж инф": 5, "Кул-дос-сп": 3},
            "weights": {"Население": 1, "Транспорт": 2, "Экология": 1, "Соц-об": 1, "Инж инф": 2, "Кул-дос-сп": 1}
        },
        "Рекреационная": {
            "criteria": {"Население": 0, "Транспорт": 0, "Экология": 0, "Соц-об": 0, "Инж инф": 0, "Кул-дос-сп": 0},
            "weights": {"Население": 1, "Транспорт": 1, "Экология": 1, "Соц-об": 1, "Инж инф": 1, "Кул-дos-сп": 1}
        },
        "Специального назначения - медицинский центр": {
            "criteria": {"Население": 4, "Транспорт": 4, "Экология": 5, "Соц-об": 0, "Инж инф": 5, "Кул-дос-сп": 0},
            "weights": {"Население": 1, "Транспорт": 2, "Экология": 2, "Соц-об": 1, "Инж инф": 2, "Кул-дос-сп": 1}
        },
        "Специального назначения - туристический кластер": {
            "criteria": {"Население": 3, "Транспорт": 3, "Экология": 5, "Соц-об": 1, "Инж инф": 3, "Кул-дос-сп": 0},
            "weights": {"Население": 1, "Транспорт": 2, "Экология": 2, "Соц-об": 1, "Инж инф": 2, "Кул-дос-сп": 1}
        },
        "Промышленная": {
            "criteria": {"Население": (3, 4), "Транспорт": 4, "Экология": 0, "Соц-об": 1, "Инж инф": 4, "Кул-дос-сп": 2},
            "weights": {"Население": 2, "Транспорт": 2, "Экология": 1, "Соц-об": 1, "Инж инф": 2, "Кул-дос-сп": 1}
        },
        "Сельско-хозяйственная": {
            "criteria": {"Население": (2, 3), "Транспорт": 3, "Экология": (4, 5), "Соц-об": 3, "Инж инф": (2, 3), "Кул-дос-сп": 2},
            "weights": {"Население": 2, "Транспорт": 1, "Экология": 2, "Соц-об": 1, "Инж инф": 1, "Кул-дос-сп": 1}
        },
        "Транспортная инженерная": {
            "criteria": {"Население": (1, 2), "Транспорт": 0, "Экология": 0, "Соц-об": 0, "Инж инф": 2, "Кул-дос-сп": 0},
            "weights": {"Население": 1, "Транспорт": 2, "Экология": 1, "Соц-об": 1, "Инж инф": 2, "Кул-дос-сп": 1}
        }
    }

    def is_criterion_satisfied(profile_value, criterion_value):
        if isinstance(profile_value, tuple):
            return profile_value[0] <= criterion_value <= profile_value[1]
        return criterion_value >= profile_value

    def calculate_exceedance(profile_value, criterion_value):
        if isinstance(profile_value, tuple):
            if profile_value[0] <= criterion_value <= profile_value[1]:
                return criterion_value - profile_value[0]
            return 0
        return max(0, criterion_value - profile_value)

    potential_scores = {}
    for profile, data in profiles.items():
        criteria = data["criteria"]
        weights = data["weights"]
        potential = sum(
            is_criterion_satisfied(criteria.get(criterion, -1), value)
            for criterion, value in criteria_values.items()
        )
        weighted_score = sum(
            is_criterion_satisfied(criteria.get(criterion, -1), value) * weights.get(criterion, 1)
            for criterion, value in criteria_values.items()
        )
        potential_scores[profile] = (potential, weighted_score)

    # Сортировка: сначала по потенциалу и взвешенной оценке, затем "Рекреационная" в конец, но выше нулевого потенциала
    ranked_profiles = sorted(potential_scores.items(), key=lambda x: (x[0] != "Рекреационная", x[1][0], x[1][1]), reverse=True)
    ranked_profiles = [item for item in ranked_profiles if item[1][0] > 0] + [item for item in ranked_profiles if item[1][0] == 0]
    
    return ranked_profiles

# Пример использования:
criteria_values = {
    "Население": 4,
    "Транспорт": 3,
    "Экология": 5,
    "Соц-об": 4,
    "Инж инф": 4,
    "Кул-дос-сп": 3
}

ranked_profiles = calculate_potential(criteria_values)
for profile, (potential, weighted_score) in ranked_profiles:
    print(f"Профиль: {profile}, Потенциал: {potential}, Взвешенная оценка: {weighted_score}")


Профиль: Жилая застройка - ИЖС, Потенциал: 6, Взвешенная оценка: 11
Профиль: Специального назначения - туристический кластер, Потенциал: 6, Взвешенная оценка: 9
Профиль: Жилая застройка - Малоэтажная, Потенциал: 5, Взвешенная оценка: 11
Профиль: Промышленная, Потенциал: 5, Взвешенная оценка: 7
Профиль: Транспортная инженерная, Потенциал: 5, Взвешенная оценка: 7
Профиль: Жилая застройка - Среднеэтажная, Потенциал: 4, Взвешенная оценка: 10
Профиль: Специального назначения - медицинский центр, Потенциал: 4, Взвешенная оценка: 5
Профиль: Сельско-хозяйственная, Потенциал: 4, Взвешенная оценка: 5
Профиль: Общественно-деловая, Потенциал: 4, Взвешенная оценка: 4
Профиль: Жилая застройка - Многоэтажная, Потенциал: 3, Взвешенная оценка: 9
Профиль: Рекреационная, Потенциал: 6, Взвешенная оценка: 6
